# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [79]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [80]:
# Create Weather DataFrame

#weather_df = pd.read_csv('weather_output.csv')
weather_df = pd.read_csv('../output_data/cities.csv')
weather_df.dropna()
weather_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [81]:
import os
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations and Fill NaN values and convert to float
locations = weather_df[["Lat", "Lng"]].astype(float)

# Store Humidity values and Fill NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)

In [84]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [85]:
# Create Filtered Weather DataFrame 
ideal_weather_df = pd.DataFrame(weather_df)

# Fit Ideal Weather Conditions based on Cloudiness, Max Temp and Wind Speed
ideal_weather_df = filtered_weather_df.loc[(filtered_weather_df["Cloudiness"] == 0)]                                               
ideal_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]
ideal_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]    
                                              
# Drop any rows with null values
ideal_weather_df.dropna()
ideal_weather_df.reset_index(inplace=True)
del ideal_weather_df["index"]
ideal_weather_df                              

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
3,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
4,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
5,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,
6,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,
7,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [86]:
# Create New Dataframe 
hotel_df = ideal_weather_df

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df
hotel_df.size

88

In [95]:
hotel_names = []


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    latitude = row["Lat"]
    longitude = row["Lng"]

    # geocoordinates
target_coordinates = f"{latitude},{longitude}"
target_search = "Hotel"
target_radius = 5000
target_type = "Hotel"
    
    
# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}    
    
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
vacation_response = requests.get(base_url, params=params)

# convert response to json
places_data = vacation_response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 32.8859566,
                    "lng": 59.220002
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.88738412989272,
                        "lng": 59.22126852989272
                    },
                    "southwest": {
                        "lat": 32.88468447010727,
                        "lng": 59.21856887010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "56d0dc98c40ff62cd5747f10c8fa81b8a6db7b33",
            "name": "Sepehr Hotel",
            "photos": [
                {
                    "height": 5760,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/105661642100574688052\">\u0627\u0645

In [92]:
for index, row in hotel_df.iterrows():
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    
    # extract results
    results = places_data['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Index 0: opuwo.
Closest hotel is Sepehr Hotel.
Retrieving Results for Index 1: ponta do sol.
Closest hotel is Sepehr Hotel.
Retrieving Results for Index 2: cidreira.
Closest hotel is Sepehr Hotel.
Retrieving Results for Index 3: morondava.
Closest hotel is Sepehr Hotel.
Retrieving Results for Index 4: vaini.
Closest hotel is Sepehr Hotel.
Retrieving Results for Index 5: nador.
Closest hotel is Sepehr Hotel.
Retrieving Results for Index 6: mogok.
Closest hotel is Sepehr Hotel.
Retrieving Results for Index 7: birjand.
Closest hotel is Sepehr Hotel.


In [93]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [94]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))